In [1]:
import pandas as pd
import numpy as np
import requests
import sys

In [2]:
glist="T:/colin/BRice/V3/genepage-insight/example_list.txt"
klist="T:/colin/BRice/V3/genepage-insight/mock_keyword_list.txt"
olist="T:/colin/BRice/V3/genepage-insight/example_list/example_list.txt"

In [43]:
def append_summary():
    with open(klist, "r") as fk:
        pheno=[]
        for line in fk:
            pheno.append(line.rstrip())
        #print(pheno)
        summary=pd.read_csv(glist, sep="\t", header=None)
        genes=list(summary[0])
        #return summary
        

        #creating knetminer genepage urls.
        network_view=[]
        keyw2 = "+OR+".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        #print(keyw2)
        #define species
        species="wheatknet"
        for i in genes:
            link="http://knetminer.rothamsted.ac.uk/{}/genepage?list={}&keyword={}".format(species, i, keyw2)
            r=requests.get(link)
            network_view.append(r.url)

        #obtaining knetscores for genes
        keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        genestr=(",").join(genes)
        link="http://knetminer.rothamsted.ac.uk/{}/genome?".format(species)
        parameters={"keyword":keyw1, "list":genestr}
        r=requests.get(link, params=parameters)

        #check if requests is successful
        if not r.ok:
                r.raise_for_status()
                sys.exit()
        
        #extract unicode string of geneTable decoded from json
        decoded=r.json()[u'geneTable'].split("\t")
        #remove space or newline at the end
        decoded=(decoded)[:-1]
        
        colnum=9
        #tabulate genetable into 9 columns.
        genetable=np.array(decoded).reshape(len(decoded)//colnum, colnum)
        genetable=pd.DataFrame(genetable[1:,:], columns=genetable[0,:])
                
        knetgenes=list(genetable[u'ACCESSION'])
        knetscores=list(genetable[u'SCORE'])
        knetchro=list(genetable[u'CHRO'])
        knetstart=list(genetable[u'START'])

        #map genes to snps via a dictionary.
        knetdict=dict(zip(knetgenes, knetscores))
        ordered_score=[]
        for i in summary[0]:
            #convert gene id to upper case to avoid sensitivity issues.
            i=i.upper()
            ordered_score.append(knetdict[u'{}'.format(i)])
        summary[u'knetscore'] = ordered_score
        summary[u'network_view']=network_view
        summary.to_csv(olist, sep="\t")
      
        return summary

In [44]:
append_summary()

0 knetscore  \
0  TRAESCS1A02G173400    306.22   
1  TRAESCS4B02G293600     13.10   
2  TRAESCS6A02G024400      0.00   
3  TRAESCS6A02G222000    104.23   
4  TRAESCS5A02G030800    227.47   
5  TRAESCS4D02G275500     40.21   
6  TRAESCS7D02G538400     76.91   
7  TRAESCS7A02G107400     88.59   
8  TRAESCS3A02G231900      0.00   
9  TRAESCS3B02G302100     35.66   

                                        network_view  
0  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
1  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
2  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
3  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
4  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
5  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
6  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
7  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
8  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
9  https://knetminer.rothamsted.ac.uk/wheatknet/g...

In [49]:
def snippet():
    with open(klist, "r") as fk:
        pheno=[]
        for line in fk:
            pheno.append(line.rstrip())
        #print(pheno)
        summary=pd.read_csv(glist, sep="\t", header=None)
        summary.rename (
        columns={
            0:"GENE"
        }, inplace=True
        )
        genes=list(summary["GENE"])
        
        #creating knetminer genepage urls.
        network_view=[]
        keyw2 = "+OR+".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        #print(keyw2)
        #define species
        species="wheatknet"
        for i in genes:
            link="http://knetminer.rothamsted.ac.uk/{}/genepage?list={}&keyword={}".format(species, i, keyw2)
            r=requests.get(link)
            network_view.append(r.url)

        #obtaining knetscores for genes
        keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        genestr=(",").join(genes)
        link="http://knetminer.rothamsted.ac.uk/{}/genome?".format(species)
        parameters={"keyword":keyw1, "list":genestr}
        r=requests.get(link, params=parameters)

        #check if requests is successful
        if not r.ok:
                r.raise_for_status()
                sys.exit()
        
        #extract unicode string of geneTable decoded from json
        decoded=r.json()[u'geneTable'].split("\t")
        #remove space or newline at the end
        decoded=(decoded)[:-1]
        
        colnum=9
        #tabulate genetable into 9 columns.
        genetable=np.array(decoded).reshape(len(decoded)//colnum, colnum)
        genetable=pd.DataFrame(genetable[1:,:], columns=genetable[0,:])
                
        knetgenes=list(genetable[u'ACCESSION'])
        knetscores=list(genetable[u'SCORE'])
        knetchro=list(genetable[u'CHRO'])
        knetstart=list(genetable[u'START'])
        #map genes to snps via a dictionary.
        knetdict=dict(zip(knetgenes, knetscores))
        print knetdict
        ordered_score=[]
        for i in genes:
            #convert gene id to upper case to avoid sensitivity issues.
            i=i.upper()
            ordered_score.append(knetdict[u'{}'.format(i)])
        summary[u'knetscore'] = ordered_score
        summary[u'network_view']=network_view
        summary.to_csv(olist, sep="\t", index=False)
        
        summary

In [50]:
snippet()

{u'TRAESCS6A02G024400': u'0.00', u'TRAESCS7A02G107400': u'88.59', u'TRAESCS3A02G231900': u'0.00', u'TRAESCS7D02G538400': u'76.91', u'TRAESCS6A02G222000': u'104.23', u'TRAESCS4B02G293600': u'13.10', u'TRAESCS4D02G275500': u'40.21', u'TRAESCS1A02G173400': u'306.22', u'TRAESCS5A02G030800': u'227.47', u'TRAESCS3B02G302100': u'35.66'}


GENE knetscore  \
0  TRAESCS1A02G173400    306.22   
1  TRAESCS4B02G293600     13.10   
2  TRAESCS6A02G024400      0.00   
3  TRAESCS6A02G222000    104.23   
4  TRAESCS5A02G030800    227.47   
5  TRAESCS4D02G275500     40.21   
6  TRAESCS7D02G538400     76.91   
7  TRAESCS7A02G107400     88.59   
8  TRAESCS3A02G231900      0.00   
9  TRAESCS3B02G302100     35.66   

                                        network_view  
0  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
1  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
2  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
3  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
4  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
5  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
6  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
7  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
8  https://knetminer.rothamsted.ac.uk/wheatknet/g...  
9  https://knetminer.rothamsted.ac.uk/wheatknet/g...

In [36]:
"TRAESCS1A02G173400" == u'TRAESCS1A02G173400'

True